In [1]:
import numpy as np
import pandas as pd

# Introduction

For this week's notebook exercises we'll be training our fundamentals of data manipulation. Note, one of my goals for the class was to ensure that everyone had seen the code for subsetting a dataframe about a thousand times within the semester. Though it may seem mundane, operations like subsetting are so essential to data wrangling, underpin so many other complex manipulations, that you can seldom get "too good" at them. The tricky part is keeping things interesting.

# Homework

Let's pretend that you work for an automobile regulatory agency. One of the key questions you have is the relationship between city gas mileage and highway gas mileage. Generally, city gas mileage is consistently lower than highway gas mileage, just given the necessity of stopping-and-starting with traffic lights, stop signs, crosswalks, etc. But *across* cars, city mileage should track closely to highway mileage, gas efficiency being an intrinsic part of a car's design.

Run the code chunk below to get your dataset (note I've tampered with this otherwise real data set for the sake of this assignment), and answer the following questions. Highway mileage is given by the variable `hwy`, and city mileage is given by the variable `cty`.

1) Let's define outliers as values that are $1.5^*IQR$ greater than the 3rd quartile, or $1.5^*IQR$ less than the 1st quartile ($IQR$ is the 3rd quartile minus the 1st quartile). Any cars that are outliers on *either* `cty` or `hwy` will be considered outliers for question 1 and 2. How many cars are outliers with regard to this definition?

2) What's the difference in correlation between highway mileage and city mileage when considering only non-outliers, and when considering all the data?

3) Now let's define outliers in terms of how strange the *relationship* between our variables of interest are. Create a new variable called `hwylesscty` that's defined as the absolute value difference between highway mileage and city mileage for each car. How many cars are outliers with regard to `hwylesscty`?

4) This time we'll define outliers based on 'residuals'. Run a linear model regressing `hwy` on `cty` using *all* the data, and calculate the residuals for each observations (i.e., the difference between the predicted `hwy` value and the observed `hwy` value). How many observations' residuals values are outliers (similar IQR-based definition as explained in Question 1)?

5) Subset the data to take out the observations with 'high' residuals (as defined in Question 4). What's the difference in correlation between highway mileage and city mileage when considering this subset versus all the data?

In [9]:
#!!!DO NOT TOUCH ANYTHING BELOW HERE!!!#
def func_datgen(pernoseq):
  np.random.seed(pernoseq)
  tempdat = pd.read_csv('https://raw.githubusercontent.com/tidyverse/ggplot2/main/data-raw/mpg.csv')
  tempdat = tempdat.loc[np.random.choice(np.arange(0, len(tempdat)), int(2*(len(tempdat)/3)), replace = False)]
  tempdat = tempdat.reset_index(drop = True)
  targetidx = np.random.choice(np.arange(0, len(tempdat)), np.random.choice(np.arange(11, 15)), replace = False)
  tempdat.loc[targetidx, 'hwy'] = np.around(tempdat.loc[targetidx, 'hwy'] + np.random.uniform(15, 20, len(targetidx)), 0)
  targetidx = np.random.choice(np.arange(0, len(tempdat)), np.random.choice(np.arange(14, 17)), replace = False)
  tempdat.loc[targetidx, 'cty'] = np.around(tempdat.loc[targetidx, 'cty'] + np.random.uniform(-7, -5, len(targetidx)), 0)
  targetidx1 = np.random.choice(np.where((tempdat['hwy'] > np.quantile(tempdat['hwy'], .75)) & (tempdat['cty'] > np.quantile(tempdat['cty'], .75)))[0], 3)
  targetidx2 = np.random.choice(np.where((tempdat['hwy'] < np.quantile(tempdat['hwy'], .25)) & (tempdat['cty'] < np.quantile(tempdat['cty'], .25)))[0], 3)
  tempdat.loc[targetidx1, 'hwy'], tempdat.loc[targetidx2, 'hwy'] = tempdat.loc[targetidx2, 'hwy'].values, tempdat.loc[targetidx1, 'hwy'].values
  return tempdat
#!!!DO NOT TOUCH ANYTHING ABOVE HERE!!!#

In [57]:
dat = func_datgen(9271)
dat

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,subaru,impreza awd,2.5,1999,4,auto(l4),4,19,26,r,subcompact
1,nissan,altima,2.4,1999,4,manual(m5),f,21,29,r,compact
2,mercury,mountaineer 4wd,4.0,2008,6,auto(l5),4,13,19,r,suv
3,subaru,forester awd,2.5,2008,4,auto(l4),4,18,23,p,suv
4,jeep,grand cherokee 4wd,4.7,2008,8,auto(l5),4,14,19,r,suv
...,...,...,...,...,...,...,...,...,...,...,...
151,nissan,maxima,3.0,1999,6,manual(m5),f,19,25,r,midsize
152,subaru,impreza awd,2.5,2008,4,manual(m5),4,20,43,r,compact
153,ford,explorer 4wd,4.0,1999,6,auto(l5),4,8,17,r,suv
154,lincoln,navigator 2wd,5.4,1999,8,auto(l4),r,11,16,p,suv


In [58]:
q1cty = np.percentile(dat['cty'], 25)
q3cty = np.percentile(dat['cty'], 75)
iqrcty = q3cty-q1cty

q1hwy = np.percentile(dat['hwy'], 25)
q3hwy = np.percentile(dat['hwy'], 75)
iqrhwy = q3hwy-q1hwy

outliers = (dat['cty'] < q1cty - (1.5 * iqrcty)) | (dat['cty'] > q3cty + (1.5 * iqrcty)) | (dat['hwy'] < q1hwy - (1.5 * iqrhwy)) | (dat['hwy'] > q3hwy + (1.5 * iqrhwy))
outliers.sum()


1

In [59]:
corr = dat['cty'].corr(dat['hwy'])
corrout = dat[~outliers]['cty'].corr(dat[~outliers]['hwy'])
corrout - corr

-0.018682613189325292

In [60]:
dat['hwylesscty'] = abs(dat['hwy'] - dat['cty'])

q1 = np.percentile(dat['hwylesscty'], 25)
q3 = np.percentile(dat['hwylesscty'], 75)
iqr = q3 - q1

low = q1 - (1.5 * iqr)
up = q3 + (1.5 * iqr)

out = (dat['hwylesscty'] < low) | (dat['hwylesscty'] > up)
out.sum()

17

In [61]:
import scipy.stats as stats
slope = stats.linregress(dat['cty'], dat['hwy'])[0]
intercept = stats.linregress(dat['cty'], dat['hwy'])[1]

dat['pred'] = intercept + (slope * dat['cty'])
dat['residuals'] = dat['hwy'] - dat['pred']

q1 = np.percentile(dat['residuals'], 25)
q3 = np.percentile(dat['residuals'], 75)
iqr = q3-q1

out = (dat['residuals'] < q1 - (1.5 * iqr)) | (dat['residuals'] > q3 + (1.5 * iqr))
out.sum()

20

In [62]:
#This code chunk demonstrates how to export your answers into a .csv file
#Fill in each part with your answers:
  #exportobj = pd.DataFrame({'PerNoSeq': ,'Question1': , 'Question2': , 'Question3': , 'Question4': , 'Question5':, 'CollaboratorNames':})
      #Note, fill in with '' if no collaborators; if multiple, type names in one '' separated with commas
exportobj = pd.DataFrame({'PerNoSeq': [9271],'Question1': [1], 'Question2': [-0.0187], 'Question3': [17], 'Question4': [20], 'Question5': [999], 'CollaboratorNames': ['']})

#Then, export your object with the code below
exportobj.to_csv("W12.csv")
    #Remember that after exporting, the file will appear in the "Files" tab (check the LHS of the screen); from there, download onto your machine, and upload it to Blackboard

Based on the personal number sequence `12345`, the answers to the above questions should be as follows:

1: 4

2: 0.0173

3: 14

4: 16

5: 0.258